In [29]:
# Core scverse libraries
import scanpy as sc
import anndata as ad
import skimage
import igraph

# Data retrieval
import pooch

sc.settings.set_figure_params(dpi=50, facecolor="white")

Importing the packages used for this project
Anndata is a file format for matrix like data, Data is in the form n_observations x n_variables. In this case, the observations are the cells measured and the variables are the expression of each gene. Simply, the matrix is cells x genes

sc.settings.set_figure_params(dpi=50, facecolor="white") configures the styling of plots used later on

In [ ]:
#Importing example data
EXAMPLE_DATA = pooch.create(
    path=pooch.os_cache("scverse_tutorials"),
    base_url="doi:10.6084/m9.figshare.22716739.v1/",
)
EXAMPLE_DATA.load_registry_from_doi()

In [ ]:
samples = {
    "s1d1": "s1d1_filtered_feature_bc_matrix.h5",
    "s1d3": "s1d3_filtered_feature_bc_matrix.h5",
}

adatas = {}
for sample_id, filename in samples.items():
    path = EXAMPLE_DATA.fetch(filename)
    sample_adata = sc.read_10x_h5(path)
    sample_adata.var_names_make_unique()
    adatas[sample_id] = sample_adata

adata = ad.concat(adatas, label="sample")
adata.obs_names_make_unique()
print(adata)

This section of code places the imported example data into an anndata object
samples dictionary containts the two samples selected for analysis. Data set contains many samples but only 2 were selected for the tutorial

Each sample is looped through, read the data. Data was in 10x genomic format so the specific read function for that format was used 
The variable names are made to be unique. This just appends integers to repeaded strings in order to make each one unique
Then this adata object is put in the adata dictionary with it's sample id as the key

Then the two adata objects are concatonated to put all of the data in a single adata object
Finally, indexs are made unique again by appending integers to repeated elements



## Quality Control

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

calculate_qc_metrics() calculates the quality control metrics 
Metrics are calculated for mitochondrial, ribosomal, and hemoglobin genes

These metrics tell us the total number of genes expressed, the total gene counts per cell, and the percentage counts of mitochondrial genes which can be used to validate the quality of the data set

These metrics are also necessary for filtering the data

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)

Using the qc data generated above we can now filter some cells. This code removes cells with less than 100 genes and removes genes that are expressed in less than 3 cells

Qc should be done per sample on data sets with many batches because qc thresholds can differ between them 

## Normalization

In [ ]:
sc.pp.scrublet(adata, batch_key="sample")

Doublets then must be identified and removed to avoid missclassification and distortions later on
scrublet identifies doublets and adds a doublet score (value) and predicted doublet (boolean) for each observed cell

Doublets can be filtered now or later using either the doublet score or predicted doublet metrics


In [ ]:
# Saving count data
adata.layers["counts"] = adata.X.copy()

In [ ]:
# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)

Data is normalized to accurately compare expression between cells
This normalization is done by a count depth scaling followed by a log plus one transformation
The count depth is the average number of reads covering a reference base

Count depths scaling normalizes the data to a specific size such as to the median count depth in the data set, or a given integer
The size is set by the target_sum property. In this case we are normalizing to the median count depth of the data set


## Feature Selection

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key="sample")

The next step is feature selection, this is done in order to reduce the dimensionality to the most informative genes
Genes with the highest variability are the most informative so the top 2000 most variable genes were selected

In [ ]:
sc.pl.highly_variable_genes(adata)

## Dimensionality Reduction

In [ ]:
sc.tl.pca(adata)

A PCA(Principal Component Analysis) reveals the main axes of variation and helps denoise the data

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=50, log=True)

Graphing tells us how many PCs to consider. In general overestimating is not detrimental though

In [ ]:
sc.pl.pca(
    adata,
    color=["sample", "sample", "pct_counts_mt", "pct_counts_mt"],
    dimensions=[(0, 1), (2, 3), (0, 1), (2, 3)],
    ncols=2,
    size=2,
)

Plotting the PCs can identify if there are any batch or qc metrics causing significant variance
Above we can see that there is not significant variance between the two sample sets (s1d1 and s1d3) so we are all good

## Nearest Neighbor Graph Construction

In [ ]:
sc.pp.neighbors(adata)

In [ ]:
sc.tl.umap(adata)

Fistly, a neighborhood graph using the PCA representation is computed. It places similar cells in groups
Then the graph is put into 2 dimensions for proper visualization  which we then plot

In [ ]:
sc.pl.umap(
    adata,
    color="sample",
    # Setting a smaller point size to get prevent overlap
    size=2,
)

## Clustering

In [ ]:
sc.pl.umap(adata, color=["leiden"])